In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import string
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Ruchit
[nltk_data]     Jain\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
df=pd.read_csv("spam_ham_dataset.csv")

In [ ]:
df


,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...,...
5166,1518,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,404,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,2933,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,1409,ham,Subject: industrial worksheets for august 2000...,0


In [ ]:
df=df.drop(columns=["Unnamed: 0","label_num"])

In [ ]:
df


,label,text
0,ham,Subject: enron methanol ; meter # : 988291\r\n...
1,ham,"Subject: hpl nom for january 9 , 2001\r\n( see..."
2,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar..."
3,spam,"Subject: photoshop , windows , office . cheap ..."
4,ham,Subject: re : indian springs\r\nthis deal is t...
...,...,...
5166,ham,Subject: put the 10 on the ft\r\nthe transport...
5167,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...
5168,ham,Subject: calpine daily gas nomination\r\n>\r\n...
5169,ham,Subject: industrial worksheets for august 2000...


In [ ]:
df['spam']=np.where(df['label']=='spam', 1,0)

In [ ]:
df

,label,text,spam
0,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,spam,"Subject: photoshop , windows , office . cheap ...",1
4,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...
5166,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,ham,Subject: industrial worksheets for august 2000...,0


In [ ]:
    df.isnull().sum()

label    0
text     0
spam     0
dtype: int64

In [ ]:
# df=df.dropna()
# df=df.drop_duplicates()

In [ ]:
print(f"Duplicated values in a dataset: {df.duplicated().sum()}")

Duplicated values in a dataset: 178


In [ ]:
def clean_text(text):

    text=''.join([char for char in text if char not in string.punctuation])
    words=text.split()
    stop_words=stopwords.words("english")
    stemmer = SnowballStemmer('english')
    words = [stemmer.stem(word) for word in words if word.lower() not in stop_words]
    return ' '.join(words)


df['clean_text'] = df['text'].apply(clean_text)
    

In [ ]:
df.head(10)['clean_text']

0    subject enron methanol meter 988291 follow not...
1    subject hpl nom januari 9 2001 see attach file...
2    subject neon retreat ho ho ho around wonder ti...
3    subject photoshop window offic cheap main tren...
4    subject indian spring deal book teco pvr reven...
5    subject ehronlin web address chang messag inte...
6    subject spring save certif take 30 save 30 use...
7    subject look medic best sourc difficult make m...
8    subject nom actual flow 2 26 agre forward meli...
9    subject nomin oct 21 23 2000 see attach file h...
Name: clean_text, dtype: object

In [ ]:
df=df.drop(columns=['text'])

In [ ]:
df

,label,spam,clean_text
0,ham,0,subject enron methanol meter 988291 follow not...
1,ham,0,subject hpl nom januari 9 2001 see attach file...
2,ham,0,subject neon retreat ho ho ho around wonder ti...
3,spam,1,subject photoshop window offic cheap main tren...
4,ham,0,subject indian spring deal book teco pvr reven...
...,...,...,...
5166,ham,0,subject put 10 ft transport volum decreas 2500...
5167,ham,0,subject 3 4 2000 follow nom hpl take extra 15 ...
5168,ham,0,subject calpin daili gas nomin juli mention ea...
5169,ham,0,subject industri worksheet august 2000 activ a...


In [ ]:
vectorizer = CountVectorizer()
X=vectorizer.fit_transform(df["clean_text"])
y=df["spam"]

In [ ]:
X_train, X_text, y_train, y_test=train_test_split(X, y ,test_size=0.2,random_state=42)

In [ ]:
# Define models
models = {
    "Naive Bayes": MultinomialNB(),
    "SVM": SVC(),
}

best_results = {}
best_model_name = None
best_accuracy = 0.0 

# Train and evaluate models
for name, model in models.items():
    model.fit(X_train, y_train)  # Training  the model
    y_pred = model.predict(X_text)  # Predict on the test set

    accuracy = accuracy_score(y_test, y_pred)  # Calculate accuracy
    best_results[name] = accuracy  # Store accuracy in the results dictionary

    # Display accuracy and classification report for the current model
    print(f"{name} Accuracy: {accuracy:.2f}")
    print("-----------------------------------------------------")
    print(classification_report(y_test, y_pred))

    # Check if this is the best accuracy so far
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model_name = name

# Display the model with the best accuracy at the end
print("\nBest Model:")
print(f"{best_model_name} with Accuracy: {best_accuracy:.2f}")

Naive Bayes Accuracy: 0.97
-----------------------------------------------------
              precision    recall  f1-score   support

           0       0.99      0.98      0.98       742
           1       0.95      0.97      0.96       293

    accuracy                           0.97      1035
   macro avg       0.97      0.97      0.97      1035
weighted avg       0.98      0.97      0.97      1035

SVM Accuracy: 0.96
-----------------------------------------------------
              precision    recall  f1-score   support

           0       0.99      0.95      0.97       742
           1       0.89      0.97      0.93       293

    accuracy                           0.96      1035
   macro avg       0.94      0.96      0.95      1035
weighted avg       0.96      0.96      0.96      1035


Best Model:
Naive Bayes with Accuracy: 0.97


In [ ]:
data = {
    'spam': ['Congratulations! You’ve won a $1,000 gift card. Click here to claim your prize']
}

In [ ]:
db1=pd.DataFrame(data)


In [ ]:
r=vectorizer.transform(db1["spam"])

In [ ]:
 ruc = model.predict(r)

In [ ]:
for email, pred in zip(db1['spam'], ruc):
    print(email, "->", "Spam" if pred==1 else "Not Spam")

Congratulations! You’ve won a $1,000 gift card. Click here to claim your prize -> Spam


In [ ]:
# Interactive UI for Email Spam Detection
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create widgets
email_input = widgets.Textarea(
    value='',
    placeholder='Enter the email content here to check if it\'s spam or not...',
    description='Email Text:',
    layout=widgets.Layout(width='100%', height='200px'),
    style={'description_width': 'initial'}
)

predict_button = widgets.Button(
    description='🔍 Check Spam',
    button_style='primary',
    layout=widgets.Layout(width='200px', margin='10px 0px')
)

result_output = widgets.HTML(
    value='<div style="padding: 10px; text-align: center; color: #666;">Enter an email above and click "Check Spam" to see the prediction</div>',
    layout=widgets.Layout(width='100%', margin='10px 0px')
)

def predict_spam(b):
    try:
        # Get the input text
        email_text = email_input.value.strip()
        
        if not email_text:
            result_output.value = '<div style="padding: 15px; background-color: #fff3cd; border: 1px solid #ffeaa7; border-radius: 5px; color: #856404;"><strong>⚠️ Warning:</strong> Please enter some email content to analyze.</div>'
            return
        
        # Clean the text using the same function from the notebook
        cleaned_email = clean_text(email_text)
        
        # Transform using the trained vectorizer
        email_vectorized = vectorizer.transform([cleaned_email])
        
        # Make prediction using the trained model
        prediction = model.predict(email_vectorized)[0]
        prediction_proba = model.predict_proba(email_vectorized)[0]
        
        # Format the result
        if prediction == 1:
            result_html = f'''
            <div style="padding: 20px; background-color: #f8d7da; border: 1px solid #f5c6cb; border-radius: 10px; text-align: center;">
                <h3 style="color: #721c24; margin: 0 0 10px 0;">🚨 SPAM DETECTED</h3>
                <p style="margin: 0; font-size: 16px; color: #721c24;">This email appears to be spam.</p>
                <p style="margin: 10px 0 0 0; font-size: 14px; color: #856404;">Confidence: {prediction_proba[1]:.2%}</p>
            </div>
            '''
        else:
            result_html = f'''
            <div style="padding: 20px; background-color: #d4edda; border: 1px solid #c3e6cb; border-radius: 10px; text-align: center;">
                <h3 style="color: #155724; margin: 0 0 10px 0;">✅ NOT SPAM</h3>
                <p style="margin: 0; font-size: 16px; color: #155724;">This email appears to be legitimate.</p>
                <p style="margin: 10px 0 0 0; font-size: 14px; color: #856404;">Confidence: {prediction_proba[0]:.2%}</p>
            </div>
            '''
        
        result_output.value = result_html
        
    except Exception as e:
        result_output.value = f'<div style="padding: 15px; background-color: #f8d7da; border: 1px solid #f5c6cb; border-radius: 5px; color: #721c24;"><strong>❌ Error:</strong> {str(e)}</div>'

# Connect button to function
predict_button.on_click(predict_spam)

# Create the UI layout
ui_container = widgets.VBox([
    widgets.HTML('<h2 style="text-align: center; color: #333; margin-bottom: 20px;">📧 Email Spam Detector</h2>'),
    email_input,
    predict_button,
    result_output
], layout=widgets.Layout(padding='20px', border='1px solid #ddd', border_radius='10px', margin='20px 0px'))

display(ui_container)